## Warm Starting BERT 2 GPT2

Bert2GPT2 is a EncoderDecoderModel, meaning that 

* the encoder -->> bert-base-uncased BERT model
* the decoder -->> GPT2 model. 

Leveraging the EncoderDecoderFramework, the two pretrained models can simply be loaded into the framework.

The decoder of an EncoderDecoder model needs cross-attention layers and usually makes use of causal masking for auto-regressiv generation. 





In [61]:

## !pip install nlp


In [62]:

import torch
import transformers
import numpy
import datasets
import pandas as pd
import nlp
import logging
import hashlib
import os

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import BertTokenizerFast
from transformers import BertTokenizer, GPT2Tokenizer
from transformers import EncoderDecoderModel, Trainer, TrainingArguments
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments




In [63]:

encoder_length = 512
decoder_length = 128
batch_size     = 16


In [64]:

logging.basicConfig(level=logging.INFO)


In [65]:


model     = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-cased", "gpt2")



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.6.crossattention.c_proj.bias', 'h.8.cro

In [66]:

# cache is currently not supported by EncoderDecoder framework

model.decoder.config.use_cache = False



* CLS token will work as BOS token
* SEP token will work as EOS token

In [67]:

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
bert_tokenizer.bos_token = bert_tokenizer.cls_token
bert_tokenizer.eos_token = bert_tokenizer.sep_token



## Make sure GPT2 appends EOS in begin and end


In [68]:


def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs


In [69]:

GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens

gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")



In [70]:

# set pad_token_id to unk_token_id -> be careful here as unk_token_id == eos_token_id == bos_token_id

gpt2_tokenizer.pad_token = gpt2_tokenizer.unk_token



## Set decoding params


In [71]:


model.config.decoder_start_token_id = gpt2_tokenizer.bos_token_id
model.config.eos_token_id           = gpt2_tokenizer.eos_token_id
model.config.max_length             = 142
model.config.min_length             = 56
model.config.no_repeat_ngram_size   = 3
model.early_stopping                = True
model.length_penalty                = 2.0
model.num_beams                     = 4



## Load train and validation data


In [72]:



train_dataset = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_dataset   = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:5%]")


In [73]:
train_dataset.info.description

'CNN/DailyMail non-anonymized summarization dataset.\n\nThere are two features:\n  - article: text of news article, used as the document to be summarized\n  - highlights: joined text of highlights with <s> and </s> around each\n    highlight, which is the target summary\n'

In [76]:

df = pd.DataFrame(   train_dataset[:3]      )

del df["id"]

for column, typ in train_dataset.features.items():
      if isinstance(typ, ClassLabel):
          df[column] = df[column].transform(lambda i: typ.names[i])
        
display(HTML(df.to_html()))


,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in ""Harry Potter and the Order of the Phoenix"" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. ""I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,"" he told an Australian interviewer earlier this month. ""I don't think I'll be particularly extravagant. ""The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."" At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film ""Hostel: Part II,"" currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. ""I'll definitely have some sort of party,"" he said in an interview. ""Hopefully none of you will be reading about it."" Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. ""People are always looking to say 'kid star goes off the rails,'"" he told reporters last month. ""But I try very hard not to go that way because it would be too easy for them."" His latest outing as the boy wizard in ""Harry Potter and the Order of the Phoenix"" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called ""My Boy Jack,"" about author Rudyard Kipling and his son, due for release later this year. He will also appear in ""December Boys,"" an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's ""Equus."" Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: ""I just think I'm going to be more sort of fair game,"" he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.",Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund .
1,"Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the ""forgotten floor,"" where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the ""forgotten floor."" Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually ""avoidable felonies."" He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman. So, they end up on the ninth floor severely mentally disturbed, but not getting any real help because they're in jail. We toured the jail with Leifman. He is well

In [77]:
train_dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [78]:
'''

>>> from nlp import load_dataset
>>> datasets = load_dataset('squad')
>>> print(datasets)
{'train': Dataset(schema: {'id': 'string', 'title': 'string', 'context': 'string', 'question': 'string', 'answers': 'struct<text: list<item: string>, answer_start: list<item: int32>>'}, num_rows: 87599),
 'validation': Dataset(schema: {'id': 'string', 'title': 'string', 'context': 'string', 'question': 'string', 'answers': 'struct<text: list<item: string>, answer_start: list<item: int32>>'}, num_rows: 10570)
}

'''

"\n\n>>> from nlp import load_dataset\n>>> datasets = load_dataset('squad')\n>>> print(datasets)\n{'train': Dataset(schema: {'id': 'string', 'title': 'string', 'context': 'string', 'question': 'string', 'answers': 'struct<text: list<item: string>, answer_start: list<item: int32>>'}, num_rows: 87599),\n 'validation': Dataset(schema: {'id': 'string', 'title': 'string', 'context': 'string', 'question': 'string', 'answers': 'struct<text: list<item: string>, answer_start: list<item: int32>>'}, num_rows: 10570)\n}\n\n"

In [79]:


'''

https://huggingface.co/docs/datasets/v0.4.0/loading_datasets.html
'''


'\n\nhttps://huggingface.co/docs/datasets/v0.4.0/loading_datasets.html\n'

## ROGUE metric 

Load rouge for validation


In [ ]:


rouge = nlp.load_metric("rouge", experiment_id=1)



In [ ]:

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = gpt2_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = gpt2_tokenizer.eos_token_id
    label_str = gpt2_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }






## map data correctly


In [37]:


def map_to_encoder_decoder_inputs(batch):    # Tokenizer will automatically set [BOS] <text> [EOS] 
    # use bert tokenizer here for encoder
    inputs = bert_tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_length)
    # force summarization <= 128
    outputs = gpt2_tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]

    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch




## Make train dataset ready

In [ ]:


train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights"],
)

train_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



## Same for validation dataset


In [ ]:


val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["article", "highlights"],
)

val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



## Set training arguments 

These params are not really tuned, feel free to change


In [ ]:


training_args = TrainingArguments(
    output_dir                     = "./",
    per_device_train_batch_size    = batch_size,
    per_device_eval_batch_size     = batch_size,
    predict_from_generate          = True,
    evaluate_during_training       = True,
    do_train                       = True,
    do_eval                        = True,
    logging_steps                  = 1000,
    save_steps                     = 1000,
    eval_steps                     = 1000,
    overwrite_output_dir           = True,
    warmup_steps                   = 2000,
    save_total_limit               = 10,
    fp16                           = True,
)


## Instantiate trainer


In [ ]:


trainer = Trainer(
    model             = model,
    args              = training_args,
    compute_metrics   = compute_metrics,
    train_dataset     = train_dataset,
    eval_dataset      = val_dataset,
)


## Start training

In [ ]:


trainer.train()


## Fully pre-trained bert2gpt2

There is also a pre-trained bert2gpt2 named bert2gpt2-cnn_dailymail-fp16.

reuse tokenizer from bert2bert encoder-decoder model

Code below

In [3]:

model    = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2gpt2-cnn_dailymail-fp16")
bert_tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16")


You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.


In [14]:

article = """(CNN)Sigma Alpha Epsilon is under fire for a video showing party-bound fraternity members singing a racist chant. SAE's national chapter suspended the students, but University of Oklahoma President David B
oren took it a step further, saying the university's affiliation with the fraternity is permanently done. The news is shocking, but it's not the first time SAE has faced controversy. SAE was founded March 9, 185
6, at the University of Alabama, five years before the American Civil War, according to the fraternity website. When the war began, the group had fewer than 400 members, of which "369 went to war for the Confede
rate States and seven for the Union Army," the website says. The fraternity now boasts more than 200,000 living alumni, along with about 15,000 undergraduates populating 219 chapters and 20 "colonies" seeking fu
ll membership at universities. SAE has had to work hard to change recently after a string of member deaths, many blamed on the hazing of new recruits, SAE national President Bradley Cohen wrote in a message on t
he fraternity's website. The fraternity's website lists more than 130 chapters cited or suspended for "health and safety incidents" since 2010. At least 30 of the incidents involved hazing, and dozens more invol
ved alcohol. However, the list is missing numerous incidents from recent months. Among them, according to various media outlets: Yale University banned the SAEs from campus activities last month after members al
legedly tried to interfere with a sexual misconduct investigation connected to an initiation rite. Stanford University in December suspended SAE housing privileges after finding sorority members attending a frat
ernity function were subjected to graphic sexual content. And Johns Hopkins University in November suspended the fraternity for underage drinking. "The media has labeled us as the 'nation's deadliest fraternity,
' " Cohen said. In 2011, for example, a student died while being coerced into excessive alcohol consumption, according to a lawsuit. SAE's previous insurer dumped the fraternity. "As a result, we are paying Lloy
d's of London the highest insurance rates in the Greek-letter world," Cohen said. Universities have turned down SAE's attempts to open new chapters, and the fraternity had to close 12 in 18 months over hazing in
cidents."""


In [15]:

input_ids  = bert_tokenizer(article, return_tensors="pt").input_ids


In [16]:

output_ids = model.generate(input_ids)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [7]:

# we need a gpt2 tokenizer for the output word embeddings


gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")



In [8]:


print(gpt2_tokenizer.decode(output_ids[0], skip_special_tokens=True))


Theresa May's visit to the US was a'very significant'
Ninth reign of the 'greatest hope abuse'
Theresa was the first to visit the US in the 'Greatest Hope Abuse'
Tunisian 'discovery' of the Great Britain's 'lost' imperial possessions.
The 'great' British imperialists were 'unjustly persecuted'
Aircraft Club 'fast hours deep' and 'deeply felt'
Comes as the US is preparing to build a new naval base in the UK.
